<!--
Doc Writer email@nixdabei.de
v0.0.1, 2021-03-23
-->
[Home](../index.ipynb) / Check controller
***
<span style="font-size:20pt;">Check controller</span>
***

# Files

## List files and show file contents
   
Windows:
Prüfen Sie die Nummer des COM-Ports mit dem Gerätemanager,   
dort in der Zeile Anschlüsse(COM & LPT) durch Öffnen des Abschnitts mit >   
Für den ESP32 steht der Eintrag: Silicon Labs CP210x ...   
Ändern Sie dann den Eintrag entsprechend.   
(Mit # wird eine Zeile auskommentiert und wird nicht als Code interpretiert.)

In [ ]:
#%serialconnect --port=COM3 --baud=115200 #Windows
%serialconnect #Linux
%ls

In [ ]:
%serialconnect --port=COM3 --baud=115200 #Windows
#%serialconnect

import os

def lsDir( strDir, showContent = False ) :
    files = os.listdir(strDir)
    print( files )
    for f in files:
        print(f)
        if showContent :
            print( "Content >>---" )
            print(open(f).read())
            print( "-----------<<\n" )

lsDir( "/", True )

## Delete files

In [ ]:
%serialconnect --port=COM3 --baud=115200 #Windows
#%serialconnect
import os

def deleteFile( strFile ) :
    try:
        print( "remove file '{}'".format( strFile ) )
        os.remove( strFile)
    except:
        pass
    
def deleteDir( strDir ) :
    files = os.listdir(strDir)
    for f in files:
        deleteFile( strDir + "/" + f )

    deleteFile( strDir )

deleteDir( "/" )


# I2C Bus scan

In [ ]:
#========================================================================
# Scan I2C-Bus
#========================================================================

#%serialconnect --port=COM3 --baud=115200
%serialconnect

import machine
import info

def scanI2c():
    if info.TYPE == "Esp8266 Croduino Nova" :
        print( "Using Esp8266 Croduino Nova" )
        PIN_SCL = 4
        PIN_SDA = 5
        i2c = machine.I2C(scl=machine.Pin(PIN_SCL), sda=machine.Pin(PIN_SDA))

    elif info.TYPE == "Esp32 NodeMCU" :
        print( "Using Esp32 NodeMCU" )
        PIN_SCL = 21
        PIN_SDA = 22
        i2c = machine.I2C(1, scl=machine.Pin(PIN_SCL), sda=machine.Pin(PIN_SDA))
        
    elif info.TYPE == "Esp32 HelTec" :
        print( "Using Esp32 HelTec" )
        PIN_SCL = 15
        PIN_SDA =  4
        i2c = machine.I2C(1, scl=machine.Pin(PIN_SCL), sda=machine.Pin(PIN_SDA))   
    else :
        print( "Unknown controller!" )


    print('Scan i2c bus...')

    devices = i2c.scan()

    if len(devices) == 0:
        print( "No I2C-bus on scl={} and sda={}".format( PIN_SCL, PIN_SDA ) )
    else:
        print('i2c devices found:',len(devices))

    for device in devices:
        print("Decimal address: ",device," | Hexa address: ",hex(device))


scanI2c()

# Tests

## Check display

In [ ]:
#========================================================================
# Check Display
#========================================================================

%serialconnect --port=COM3 --baud=115200
#%serialconnect

import display

# using default address 0x3C
display = display.Display()
display.fill(0)
#display.invert(1)      # display inverted
display.invert( 0 )
display.text("OLED 128x64",  0,  0, 1) # last parameter: 1: white (default), 0: black
display.hline(0, 10, 128, 1)
display.text("Drueben am Walde", 0, 16 )   # white is default
display.text("Kaengt ein Guruh", 0, 26 ) 
display.text("Warte nur balde",  0, 36 )
display.text("Kaengurst",        0, 46 )
display.text("auch du    (J.R)", 0, 56 ) # 56: maximal
display.fill_rect(64, 56, 7, 7, 1)

display.show()

In [ ]:
#========================================================================
# Check internal orange LED
#========================================================================

#%serialconnect --port=COM3 --baud=115200 #Windows
%serialconnect

import machine
import info

LED_ONBOARD = None

if info.TYPE == "Esp8266 Croduino Nova" :
    LED_ONBOARD = 13
elif info.TYPE == "Esp32 NodeMCU" :
    print( "Sorry: 'Esp32 NodeMCU' has no usable on board LED." )
elif info.TYPE == "Esp32 HelTec" :
    LED_ONBOARD = 25
else :
    print( "Unknown controller!" )
    
def pulse(l, t):
    import time, math
    for i in range(20):
        l.duty(int(math.sin(i / 10 * math.pi) * 500 + 500))
        time.sleep_ms(t)

if  LED_ONBOARD != None :
    import machine
    led = machine.PWM(machine.Pin(LED_ONBOARD), freq=1000)
    for i in range(10):
        pulse(led, 50)
